In [1]:
#Mounting Google Drive from Google Colab
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#Changing the current working directory to the Google Drive
#%cd /content/drive/My Drive/MLDL2024_project1-Enrico

In [3]:
#Importing the necessary libraries
import os
import torch
import numpy as np
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from datasets.cityscapes import CityscapesCustom
from models.deeplabv2.deeplabv2 import get_deeplab_v2
from models.bisenet.build_bisenet import BiSeNet
from train import train_model
from utils import test_latency_FPS
from utils import test_FLOPs_params

In [4]:
#Set device agnostic code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
#Create Dataloaders for Cityscapes
cityscapes_dir = os.path.dirname(os.getcwd()) + '/Cityscapes/Cityspaces/'

cityscapes_train_dataset = CityscapesCustom(cityscapes_dir, 'train', height=8, width=16)
cityscapes_test_dataset = CityscapesCustom(cityscapes_dir, 'val', height=8, width=16)

cityscapes_train_dataloader = DataLoader(cityscapes_train_dataset, batch_size=4, shuffle=True)
cityscapes_test_dataloader = DataLoader(cityscapes_test_dataset, batch_size=4, shuffle=False)

print(f'Train: {len(cityscapes_train_dataset)} images, divided into {len(cityscapes_train_dataloader)} batches of size {cityscapes_train_dataloader.batch_size}')
print(f'Test: {len(cityscapes_test_dataset)} images, divided into {len(cityscapes_test_dataloader)} batches of size {cityscapes_test_dataloader.batch_size}')

Train: 1572 images, divided into 393 batches of size 4
Test: 500 images, divided into 125 batches of size 4


In [6]:
#Testing Classic semantic segmentation network

#Set up the model with the pretrained weights
DeepLabV2_model = get_deeplab_v2().to(device)

#Set up the loss function and the optimizer
DeepLabV2_criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
DeepLabV2_optimizer = optim.Adam(DeepLabV2_model.parameters(), lr=1e-3)

#Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

Deeplab pretraining loading...


In [7]:
#Train the model
train_model(DeepLabV2_model, DeepLabV2_criterion, DeepLabV2_optimizer,
            cityscapes_train_dataloader, cityscapes_test_dataloader,
            device, n_epochs=10, lr_schedule=False)

#Test latency and FPS
#print(test_latency_FPS(DeepLabV2_model, height=4, width=8, device=device))

#Test FLOPs and number of parameters
#print(test_FLOPs_params(DeepLabV2_model, height=4, width=8, device=device))

Epoch 1/10, Train IoU: 48.93% (867/1772), Test IoU: 37.78% (334/884)  12s]
Epoch 2/10, Train IoU: 55.58% (937/1686), Test IoU: 58.37% (516/884)  12s]
Epoch 3/10, Train IoU: 54.48% (918/1685), Test IoU: 72.74% (643/884)  12s]
Epoch 4/10, Train IoU: 65.41% (1106/1691), Test IoU: 66.40% (587/884)  12s]
Epoch 5/10, Train IoU: 61.41% (1103/1796), Test IoU: 69.12% (611/884)  12s]
Epoch 6/10, Train IoU: 68.82% (1053/1530), Test IoU: 74.66% (660/884)  13s]
Epoch 7/10, Train IoU: 71.40% (1236/1731), Test IoU: 80.20% (709/884)  12s]
Epoch 8/10, Train IoU: 73.91% (1156/1564), Test IoU: 73.42% (649/884)  12s]
Epoch 9/10, Train IoU: 72.22% (1222/1692), Test IoU: 73.98% (654/884)  12s]
Epoch 10/10, Train IoU: 71.97% (1091/1516), Test IoU: 75.11% (664/884)  12s]
Best IoU: 80.20% at epoch 7


In [8]:
#Testing Real-Time semantic segmentation network

#Set up the model with the pretrained weights
BiSeNet_model = BiSeNet(num_classes=19, context_path='resnet18').to(device)

#Set up the loss function and the optimizer for BiSeNet
BiSeNet_criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
BiSeNet_optimizer = torch.optim.SGD(BiSeNet_model.parameters(), lr=2.5e-2, momentum=0.9, weight_decay=1e-4)

#Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [9]:
#Train the model
train_model(BiSeNet_model, BiSeNet_criterion, BiSeNet_optimizer,
            cityscapes_train_dataloader, cityscapes_test_dataloader,
            device, n_epochs=10, lr_schedule=True)

#Test latency and FPS
#print(test_latency_FPS(BiSeNet_model, height=4, width=8, device=device))

#Test FLOPs and number of parameters
#print(test_FLOPs_params(BiSeNet_model, height=4, width=8, device=device))

Epoch 1/10, Train IoU: 27.88% (494/1772), Test IoU: 46.83% (414/884)  7s]
Epoch 2/10, Train IoU: 37.90% (639/1686), Test IoU: 43.55% (385/884)  7s]
Epoch 3/10, Train IoU: 34.72% (585/1685), Test IoU: 42.19% (373/884)  7s]
Epoch 4/10, Train IoU: 36.01% (609/1691), Test IoU: 46.27% (409/884)  7s]
Epoch 5/10, Train IoU: 34.69% (623/1796), Test IoU: 47.29% (418/884)  7s]
Epoch 6/10, Train IoU: 43.01% (658/1530), Test IoU: 43.55% (385/884)  7s]
Epoch 7/10, Train IoU: 35.12% (608/1731), Test IoU: 46.61% (412/884)  7s]
Epoch 8/10, Train IoU: 39.77% (622/1564), Test IoU: 47.29% (418/884)  7s]
Epoch 9/10, Train IoU: 45.51% (770/1692), Test IoU: 45.25% (400/884)  8s]
Epoch 10/10, Train IoU: 43.21% (655/1516), Test IoU: 43.44% (384/884)  8s]
Best IoU: 47.29% at epoch 5
